In [1]:
import numpy as np
import pandas as pd
from senticnet5 import senticnet
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords,wordnet
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
import re
from scipy import sparse
stop_words = set(stopwords.words('english'))
wordnet_lemmatizer = WordNetLemmatizer()

In [2]:
df = pd.read_excel('hand8_k_random.xlsx')
print(len(df))

8501


In [3]:
for cl in df.columns:
    if(cl=='Text'):
        continue
    print(df[cl].value_counts())

0    6095
1    2406
Name: Joy, dtype: int64
0    4375
1    4126
Name: Sadness, dtype: int64
0    7512
1     989
Name: Anger, dtype: int64
0    7121
1    1380
Name: Disgust, dtype: int64
0    7923
1     578
Name: Admiration, dtype: int64
0    7877
1     624
Name: Surprise, dtype: int64
0    6367
1    2134
Name: Interest, dtype: int64
0    7827
1     674
Name: Fear, dtype: int64


In [4]:
singleword=[]
for key,val in senticnet.items():
    if(len(key.split('_'))==1):
        singleword.append(key)
print(len(singleword))
word=[]
primary=[]
sec=[]
pola=[]
for x in singleword:
    word.append(x)
    primary.append(senticnet[x][4])
    sec.append(senticnet[x][5])
    pola.append(senticnet[x][7])
df_emo=pd.DataFrame(list(zip(word,primary,sec,pola)),columns=["Word","Primary","Secondary","Polarity"])
df_emo.sample(10)

39891


,Word,Primary,Secondary,Polarity
5168,categorize,#sadness,#disgust,-0.88
10796,easygoing,#joy,#admiration,0.78
32324,situation,#interest,#admiration,0.783
32373,sketchy,#surprise,#admiration,0.944
12247,exotic,#joy,#interest,0.773
16720,hydra-headed,#anger,#disgust,-0.97
15882,heck,#sadness,#disgust,-0.15
24570,operator,#interest,#admiration,0.142
28974,reappoint,#interest,#admiration,0.788
30276,roman,#interest,#admiration,0.021


In [5]:
from sklearn.model_selection import train_test_split, KFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multioutput import ClassifierChain

In [6]:
def remove_count_user_mentions(tweet):
    tweet_mentions_removed = re.subn(r'@[A-Za-z0-9]+','',tweet)
    tweet = tweet_mentions_removed[0]
    no_user_mentions = tweet_mentions_removed[1]
    return tweet,no_user_mentions
#%%
def remove_count_urls(tweet):
    tweet_url_removed = re.subn('https?://[A-Za-z0-9./]+','',tweet)
    tweet = tweet_url_removed[0]
    no_urls = tweet_url_removed[1]
    return tweet,no_urls
#%%
def remove_count_hashtags(tweet):
    no_hashtags = len({tag.strip("#") for tag in tweet.split() if tag.startswith("#")})
    tweet = re.sub("[^a-zA-Z]", " ",tweet)
    return tweet,no_hashtags    
def get_pos(word):
    tag = pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)

In [7]:
need = ["J","N","V","R"]
#need = ["V"]
neg = ["n't","not"]
punct = [".",",","?",";","!"]
opposite = {}
opposite["#joy"] = "#sadness"
opposite[0] = 1
opposite["#sadness"] = "#joy"
opposite[1] = 0
opposite["#admiration"] = "#anger"
opposite[4] = 2
opposite["#anger"] = "#admiration"
opposite[2] = 4
opposite["#surprise"] = "#fear"
opposite[5] = 7
opposite["#fear"] = "#surprise"
opposite[7] = 5
opposite["#interest"] = "#disgust"
opposite[6] = 3
opposite["#disgust"] = "#interest"
opposite[3] = 6

In [8]:
negatives = []
def normal_algo(sen):
    NEGATION_ADVERBS = ["no", "without", "nil","not", "n't", "never", "none", "neith", "nor", "non"]
    NEGATION_VERBS = ["deny", "reject", "refuse", "subside", "retract", "non"]
    CONJUCTION_WORDS = ["for", "and", "nor", "but", "or", "yet", "so"]
    sen = sen.lower()
    sen,removed_user_cnt = remove_count_user_mentions(sen)
    sen,removed_url_cnt = remove_count_urls(sen)
    sen,removed_hashtag_cnt = remove_count_hashtags(sen)
    #print(sen)
    tokens = word_tokenize(sen)
    lem = [wordnet_lemmatizer.lemmatize(t,get_pos(t)) for t in tokens]
    #print(lem)
    lem_lookup = {}
    for i in range(len(tokens)):
        lem_lookup[tokens[i]]=lem[i]
    mark_neg = {}
    nflag = False
    for t in lem:
        if(t[0] in punct or t in CONJUCTION_WORDS):
            nflag=False
        if(nflag==True):
            mark_neg[t]=1
            negatives.append(t)
        if(t in NEGATION_ADVERBS or t in NEGATION_VERBS):
            nflag=True
    tag1 = pos_tag(tokens)
    #print(tag1)
    tokens.clear()
    for x in tag1:
        #print(x)
        if(x[1][0] in need):
            tokens.append(x[0])
    val = {}
    #print(tokens)
    ret_str = ""
    for t in tokens:
        t=lem_lookup[t]
        ret_str+=t
        ret_str+=" "
        """
        if(t in senticnet):
            x = senticnet[t][4]
            #print(t)
            if(t in mark_neg):
                #print(t)
                x=opposite[x]
                #print(t,x)
            if(x in val):
                val[x]+=1
            else:
                val[x]=1
        """
    #print(mark_neg)
    return ret_str

In [9]:
analysed = [normal_algo(txt) for txt in df['Text']]
negatives = set(negatives)
qmark = []
exmark = []
f=0
for txt in df['Text']:
    f=1
    for lt in txt:
        if(lt=='?'):
            qmark.append(1)
            f=0
            break
    if(f==1):
        qmark.append(0)
    f=1
    for lt in txt:
        if(lt=='!'):
            exmark.append(1)
            f=0
            break
    if(f==1):
        exmark.append(0)
print(len(qmark))
print(len(exmark))

8501
8501


In [10]:
print(pd.Series(qmark).value_counts())
print(pd.Series(exmark).value_counts())

0    7425
1    1076
dtype: int64
0    6043
1    2458
dtype: int64


In [11]:
df['Analysed'] = analysed
df['qmark'] = qmark
df['exmark'] = exmark
df.head(10)

,Text,Joy,Sadness,Anger,Disgust,Admiration,Surprise,Interest,Fear,Analysed,qmark,exmark
0,So much for sleeping in.,0,0,0,0,0,0,0,1,so much sleep,0,0
1,College days are loooong days.. 3 more hours,0,1,0,0,0,0,1,0,college day be loooong day more hour,0,0
2,@daihard I'm headed to Kentucky this time. Nev...,0,0,0,0,0,0,1,0,i m head kentucky time never be be fun gqz,1,1
3,hella tired.. where is gilbert for the usual b...,0,0,0,0,0,0,1,0,hella tire be gilbert usual basketball talk,1,1
4,Not as dry this morning as would have liked l...,0,1,0,1,0,0,0,0,not dry morning a have like lot moisture dune ...,0,1
5,@lil_laura_loo Really? I think we have some! I...,0,1,0,0,0,0,0,1,laura loo really i think have i ve take pirite...,1,1
6,@Do512_Kristin it's a good thing they give you...,0,0,0,1,0,0,0,0,kristin s good thing give xanax something i ba...,0,0
7,"PB&amp;J, Owl City, and boredom.",0,1,0,1,0,0,0,0,pb amp j owl city boredom,0,0
8,So it's Saturday again &amp; what do I do..? W...,0,1,0,0,0,0,0,0,s saturday again amp do i do work again course,1,0
9,"trying to relax and watch Nascar, Difficult 'c...",0,1,0,1,0,0,0,0,try relax watch nascar difficult cause child d...,0,0


In [12]:
def convtodec(x):
    val = 128
    ret = 0
    for y in x:
        if(y):
            ret+=val
        val=val>>1
    return ret
labelpowerset = []
for row in df.iterrows():
    tmp = row[1][1:9].tolist()
    labelpowerset.append(convtodec(tmp))
print(len(labelpowerset))

8501


In [13]:
cntpowerset = {}
for val in labelpowerset:
    if(val in cntpowerset):
        cntpowerset[val]+=1
    else:
        cntpowerset[val]=1

cntpowersetlist = []
for key,val in cntpowerset.items():
    cntpowersetlist.append((val,key))
cntpowersetlist.sort(reverse=True)
print(len(cntpowersetlist))

95


In [14]:
label = {}
revlabel = {}
cnt = 0
for val in cntpowersetlist:
    label[val[1]] = cnt
    revlabel[cnt] = val[1]
    cnt+=1
powset = [label[x] for x in labelpowerset]
df['powerset'] = powset

In [15]:
df['powerset'].value_counts()

0     1870
1      998
2      772
3      711
4      348
      ... 
85       1
93       1
86       1
94       1
91       1
Name: powerset, Length: 95, dtype: int64

In [16]:
df_tmp = df[df['powerset']<=20]
print(len(df_tmp))

7395


In [17]:
def get_senticnet(word,em):
    em = '#'+em.lower()
    if(senticnet[word][4]==em or senticnet[word][5]==em):
        return 1
    else:
        return 0

In [18]:
X = df_tmp[['Analysed','qmark','exmark']]
Y = df_tmp['powerset']

In [19]:
vectorizer = TfidfVectorizer()
vectorizer.fit(df_tmp['Analysed'])
print(len(vectorizer.vocabulary_))

9848


In [20]:
kf = KFold(n_splits = 10)
kf.get_n_splits(X)
print(kf)

KFold(n_splits=10, random_state=None, shuffle=False)


In [21]:
def evaluation(score_list,predict_score_list):
    filter_corr = []
    exmatch = 0
    atleast1 = 0
    md1 = 0
    one_f = 0
    more_f = 0
    zero_f = 0
    sm = 0
    sdensity = 0
    hammval = 0
    test_len = len(predict_score_list[0])
    for j in range(test_len):
        cnt=0
        for i in range(8):
            hammval+=(score_list[i][j] ^ int(predict_score_list[i][j]))
            if(score_list[i][j]==1):
                cnt+=1
                sm+=1
        sdensity+=cnt/8
        if(cnt==0):
            zero_f+=1
        if(cnt==1):
            one_f+=1
        if(cnt>1):
            more_f+=1
        for i in range(8):
            mf = True
            if(int(predict_score_list[i][j])!=score_list[i][j]):
                mf=False
                break
        if(mf==True):
            exmatch+=1
            filter_corr.append(j)
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                atleast1+=1
                break
        mf = False
        for i in range(8):
            if(int(predict_score_list[i][j])==score_list[i][j] and score_list[i][j]==1):
                if(mf==True):
                    md1+=1
                    filter_corr.append(j)
                    break
                mf=True
    #print("Label Cardinality: "+ str(sm/test_len))
    #print("Label Density: "+ str(sdensity/test_len))
    print("Hamming Loss: "+str(hammval/(test_len*8)))
    hamlos = hammval/(test_len*8)
    print("Exact Prediction: "+str(exmatch/test_len))
    sub_accu = exmatch/test_len
    #print("At least one label predicted: "+str(atleast1/(test_len-zero_f)))
    #print("More than one label predicted: "+str(md1/more_f))
    tp_sum = 0
    fp_sum = 0
    fn_sum = 0
    macro_preci = 0
    macro_recall = 0
    macro_f1 = 0
    for i in range(len(score_list)):
        tmp = confusion_matrix(score_list[i],predict_score_list[i])
        tp_sum+=tmp[0][0]
        fp_sum+=tmp[0][1]
        fn_sum+=tmp[1][0]
        macro_preci_tmp=tmp[0][0]/(tmp[0][0]+tmp[0][1])
        macro_recall_tmp=tmp[0][0]/(tmp[0][0]+tmp[1][0])
        macro_f1 += ((2*macro_preci_tmp*macro_recall_tmp)/(macro_preci_tmp+macro_recall_tmp))
        macro_preci+=macro_preci_tmp
        macro_recall+=macro_recall_tmp
        #print(macro_f1)
    micro_preci = tp_sum/(tp_sum+fp_sum)
    micro_recall = tp_sum/(tp_sum+fn_sum)
    micro_f1 = (2*micro_preci*micro_recall)/(micro_preci+micro_recall)
    macro_preci/=8
    macro_recall/=8
    macro_f1/=8
    #print(micro_preci,micro_recall,micro_f1)
    #print(macro_preci,macro_recall,macro_f1)
    print("Macro F-Score: "+str(macro_f1))
    print("Micro F-Score: "+str(micro_f1))
    col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
    tmp = 0
    for i in range(len(score_list)):
        score = accuracy_score(score_list[i],predict_score_list[i]) 
        #print(col_names[i]+" accuracy: "+str(score))
        tmp += score
    print("Average Accuracy: " + str(tmp/8))
    avg_accu = tmp/8
    return (hamlos,sub_accu,macro_f1,micro_f1,avg_accu)

In [22]:
def convert2bin(xx):
    ret = []
    for i in range(8):
        if(xx & (1<<i)):
            ret.append(1)
        else:
            ret.append(0)
    #print(xx)
    ret.reverse()
    #print(ret)
    return ret

In [23]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    clf = RandomForestClassifier(n_estimators=300)
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = Y.iloc[train_index].tolist(),Y.iloc[test_index].tolist()
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    tmp_list = []
                    for cl in col_names:
                        tmp_list.append(get_senticnet(t,cl))
                    y_train.append(label[convtodec(tmp_list)])
    for word in negatives:
        if(word in senticnet):
            x_train_analysed.append("not "+word)
            x_train_qmark.append(0)
            x_train_exmark.append(0)
            tmp_list = []
            for cl in col_names:
                tmp_list.append(get_senticnet(word,cl))
            tmp_list2 = []
            for i in range(8):
                tmp_list2.append(0)
            for i in range(8):
                if(tmp_list[i]==1):
                    tmp_list2[opposite[i]] = 1
            y_train.append(label[convtodec(tmp_list2)])
    x_train_analysed_vec = vectorizer.transform(x_train_analysed)
    x_test_analysed_vec = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed_vec,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    tmp = sparse.hstack((x_test_analysed_vec,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    score_list = []
    predict_score_list = []
    for i in range(len(y_test)):
        score_list.append(convert2bin(revlabel[y_test[i]]))
        predict_score_list.append(convert2bin(revlabel[y_pred[i]]))
    np_score_list = np.array(score_list)
    transpose = np_score_list.T
    score_list = transpose.tolist()

    np_predict_score_list = np.array(predict_score_list)
    transpose = np_predict_score_list.T
    predict_score_list = transpose.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(8291, 9850) (740, 9850)
(8291,) (740,)
Hamming Loss: 0.1445945945945946
Exact Prediction: 0.33918918918918917
Macro F-Score: 0.8899636146475027
Micro F-Score: 0.9151467089611418
Average Accuracy: 0.8554054054054053


k_fold validation: 2
(8281, 9850) (740, 9850)
(8281,) (740,)
Hamming Loss: 0.14831081081081082
Exact Prediction: 0.33513513513513515
Macro F-Score: 0.8877695479528243
Micro F-Score: 0.9131037212984957
Average Accuracy: 0.8516891891891891


k_fold validation: 3
(8291, 9850) (740, 9850)
(8291,) (740,)
Hamming Loss: 0.15320945945945946
Exact Prediction: 0.3027027027027027
Macro F-Score: 0.8869642557721665
Micro F-Score: 0.9095261845386533
Average Accuracy: 0.8467905405405406


k_fold validation: 4
(8317, 9850) (740, 9850)
(8317,) (740,)
Hamming Loss: 0.1516891891891892
Exact Prediction: 0.31216216216216214
Macro F-Score: 0.8853746275512212
Micro F-Score: 0.9107178365480215
Average Accuracy: 0.8483108108108107


k_fold validation: 5
(8294, 9850) (740, 985

In [24]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))


In [25]:
df_rfc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_rfc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.144595,0.339189,0.889964,0.915147,0.855405
1,2,0.148311,0.335135,0.887770,0.913104,0.851689
2,3,0.153209,0.302703,0.886964,0.909526,0.846791
3,4,0.151689,0.312162,0.885375,0.910718,0.848311
4,5,0.150338,0.312162,0.886738,0.911759,0.849662
5,6,0.152909,0.300406,0.886779,0.910086,0.847091
6,7,0.147835,0.304465,0.889200,0.912687,0.852165
7,8,0.154601,0.312585,0.884932,0.909253,0.845399
8,9,0.146651,0.330176,0.889064,0.913637,0.853349
9,10,0.150034,0.316644,0.885698,0.911768,0.849966


In [26]:
col_names = ['Joy','Sadness','Anger','Disgust','Admiration','Surprise','Interest','Fear']
hamm_score = []
subset_accu = []
macro_f1 = []
micro_f1 = []
avg_accu = []
cnt = 1
for train_index,test_index in kf.split(X):
    clf = SVC()
    x_train,x_test = X.iloc[train_index],X.iloc[test_index]
    y_train,y_test = Y.iloc[train_index].tolist(),Y.iloc[test_index].tolist()
    print("k_fold validation: " + str(cnt))
    cnt+=1
    x_train_analysed = x_train['Analysed'].tolist()
    x_train_qmark = x_train['qmark'].tolist()
    x_train_exmark = x_train['exmark'].tolist()
    x_test_analysed = x_test['Analysed'].tolist()
    x_test_qmark = x_test['qmark'].tolist()
    x_test_exmark = x_test['exmark'].tolist()
    pre = {}
    for sen in x_train_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            pre[t]=1
    for sen in x_test_analysed:
        tok = word_tokenize(sen)
        for t in tok:
            if(t in pre):
                continue
            else:
                if(t in senticnet):
                    x_train_analysed.append(t)
                    x_train_qmark.append(0)
                    x_train_exmark.append(0)
                    tmp_list = []
                    for cl in col_names:
                        tmp_list.append(get_senticnet(t,cl))
                    y_train.append(label[convtodec(tmp_list)])
    for word in negatives:
        if(word in senticnet):
            x_train_analysed.append("not "+word)
            x_train_qmark.append(0)
            x_train_exmark.append(0)
            tmp_list = []
            for cl in col_names:
                tmp_list.append(get_senticnet(t,cl))
            tmp_list2 = []
            for i in range(8):
                tmp_list2.append(0)
            for i in range(8):
                if(tmp_list[i]==1):
                    tmp_list2[opposite[i]] = 1
            y_train.append(label[convtodec(tmp_list2)])
    x_train_analysed_vec = vectorizer.transform(x_train_analysed)
    x_test_analysed_vec = vectorizer.transform(x_test_analysed)
    tmp = sparse.hstack((x_train_analysed_vec,np.array(x_train_qmark)[:,None]))
    x_train = sparse.hstack((tmp,np.array(x_train_exmark)[:,None]))
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    tmp = sparse.hstack((x_test_analysed_vec,np.array(x_test_qmark)[:,None]))
    x_test = sparse.hstack((tmp,np.array(x_test_exmark)[:,None]))
    
    print(x_train.shape,x_test.shape)
    print(y_train.shape,y_test.shape)
    
    clf.fit(x_train,y_train)
    y_pred = clf.predict(x_test)
    score_list = []
    predict_score_list = []
    for i in range(len(y_test)):
        score_list.append(convert2bin(revlabel[y_test[i]]))
        predict_score_list.append(convert2bin(revlabel[y_pred[i]]))
    np_score_list = np.array(score_list)
    transpose = np_score_list.T
    score_list = transpose.tolist()

    np_predict_score_list = np.array(predict_score_list)
    transpose = np_predict_score_list.T
    predict_score_list = transpose.tolist()
    
    ret = evaluation(score_list,predict_score_list)
    hamm_score.append(ret[0])
    subset_accu.append(ret[1])
    macro_f1.append(ret[2])
    micro_f1.append(ret[3])
    avg_accu.append(ret[4])
    print('\n')
print('Final Result: ')
print('Average Hamming Loss: '+str(sum(hamm_score)/len(hamm_score)))
print('Average Subset Accuracy: '+str(sum(subset_accu)/len(subset_accu)))
print('Average Macro F-score: '+str(sum(macro_f1)/len(macro_f1)))
print('Average Micro F-score: '+str(sum(micro_f1)/len(micro_f1)))
print('Average of Average Accuracy: '+str(sum(avg_accu)/len(avg_accu)))

k_fold validation: 1
(8291, 9850) (740, 9850)
(8291,) (740,)
Hamming Loss: 0.1464527027027027
Exact Prediction: 0.327027027027027
Macro F-Score: 0.8798959172213772
Micro F-Score: 0.9139112302651176
Average Accuracy: 0.8535472972972973


k_fold validation: 2
(8281, 9850) (740, 9850)
(8281,) (740,)
Hamming Loss: 0.15371621621621623
Exact Prediction: 0.3324324324324324
Macro F-Score: 0.877362330176236
Micro F-Score: 0.9097222222222222
Average Accuracy: 0.8462837837837839


k_fold validation: 3
(8291, 9850) (740, 9850)
(8291,) (740,)
Hamming Loss: 0.15675675675675677
Exact Prediction: 0.29324324324324325
Macro F-Score: 0.8772920302613761
Micro F-Score: 0.9074591144794576
Average Accuracy: 0.8432432432432432


k_fold validation: 4
(8317, 9850) (740, 9850)
(8317,) (740,)
Hamming Loss: 0.14560810810810812
Exact Prediction: 0.3310810810810811
Macro F-Score: 0.8810545159382425
Micro F-Score: 0.9141776184786938
Average Accuracy: 0.8543918918918919


k_fold validation: 5
(8294, 9850) (740, 9850)


In [27]:
labels = [i+1for i in range(10)]
labels.append('average')
hamm_score.append(sum(hamm_score)/len(hamm_score))
subset_accu.append(sum(subset_accu)/len(subset_accu))
macro_f1.append(sum(macro_f1)/len(macro_f1))
micro_f1.append(sum(micro_f1)/len(micro_f1))
avg_accu.append(sum(avg_accu)/len(avg_accu))

In [28]:
df_svc = pd.DataFrame(list(zip(labels,hamm_score,subset_accu,macro_f1,micro_f1,avg_accu)),
              columns = ['k-fold','Hamming loss','Subset accuracy','Macro F-score','Micro F-score','Average Accuracy'])
df_svc

,k-fold,Hamming loss,Subset accuracy,Macro F-score,Micro F-score,Average Accuracy
0,1,0.146453,0.327027,0.879896,0.913911,0.853547
1,2,0.153716,0.332432,0.877362,0.909722,0.846284
2,3,0.156757,0.293243,0.877292,0.907459,0.843243
3,4,0.145608,0.331081,0.881055,0.914178,0.854392
4,5,0.152872,0.313514,0.878564,0.910245,0.847128
5,6,0.154939,0.317997,0.876934,0.908765,0.845061
6,7,0.156461,0.281461,0.875832,0.907657,0.843539
7,8,0.150710,0.343708,0.880615,0.911511,0.849290
8,9,0.148173,0.330176,0.881242,0.912714,0.851827
9,10,0.151049,0.334235,0.876455,0.911011,0.848951
